# Import

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Data Preparation

In [4]:

# Load the data
data = pd.read_csv('data_06_23_to_06_24.csv')

In [5]:
# Convert date column to datetime
data['date'] = pd.to_datetime(data['date'])

# Set date as index
data.set_index('date', inplace=True)

In [6]:
# Select relevant features (excluding the target variable)
features = data.drop(columns=['temperature_2m'])

In [7]:
# Target variable
target = data['temperature_2m']

In [8]:
# Scale the features and target
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

features_scaled = scaler_features.fit_transform(features)
target_scaled = scaler_target.fit_transform(target.values.reshape(-1, 1))

In [9]:
# Create a function to prepare the data for time series forecasting
def create_dataset(features, target, time_step=1):
    X, y = [], []
    for i in range(len(features) - time_step):
        X.append(features[i:(i + time_step), :])
        y.append(target[i + time_step])
    return np.array(X), np.array(y)

In [10]:
# Define the time step (e.g., 24 hours for a daily prediction)
time_step = 24

# Create the dataset
X, y = create_dataset(features_scaled, target_scaled, time_step)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training

In [11]:
# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(time_step, X_train.shape[2])),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dense(25),
    tf.keras.layers.Dense(1)
])

c:\Users\florent\_python_environments\orchestration_venv_v2\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [13]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

Epoch 1/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - loss: nan - val_loss: nan
Epoch 2/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: nan - val_loss: nan
Epoch 3/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: nan - val_loss: nan
Epoch 4/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: nan - val_loss: nan
Epoch 5/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: nan - val_loss: nan
Epoch 6/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - loss: nan - val_loss: nan
Epoch 7/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: nan - val_loss: nan
Epoch 8/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: nan - val_loss: nan
Epoch 9/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: nan - val_loss: nan
Epoch 10/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - loss: nan - val_loss: nan
Epoch 11/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: nan - val_loss: nan
Epoch 12/20
233/233 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - loss: nan - val_loss: nan
Epoch 13/20
233/233 ━━━━━

# Evaluation

In [8]:
# Predict on the test set
y_pred_scaled = model.predict(X_test)

In [ ]:
# Inverse transform the predictions and the actual values
y_pred = scaler_target.inverse_transform(y_pred_scaled)
y_test_actual = scaler_target.inverse_transform(y_test)

In [ ]:
# Calculate evaluation metrics
mse = mean_squared_error(y_test_actual, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_actual, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared Score: {r2}")

In [ ]:
# Plot actual vs predicted values
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 5))
plt.plot(y_test_actual, label='Actual Temperature')
plt.plot(y_pred, label='Predicted Temperature')
plt.legend()
plt.title('Actual vs Predicted Temperature')
plt.show()